<a href="https://colab.research.google.com/github/X-ray-Dawgz/XRayDawgz/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ls

CNN_COVID19.ipynb             Test/
CNN_COVID20.ipynb             XRayDawgz/
CNN_test_0312.ipynb           change_size.ipynb
CNN_test_3_convolution.ipynb  cut_image/
Image_Preprocess.py           docs/
Images/                       environment.yml
LICENSE                       image_prepare.py
README.md                     img_prepare.ipynb


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing import image

import numpy as np
import os

Using TensorFlow backend.
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hsiaoyuhsien/opt

In [5]:
# image size
img_width, img_height = 379,288
# train and test sample sizes
train_samples = 665
test_samples = 98
# set weights
epochs = 75
batch_size = 20
# Directories of train and test data
train_dir = 'cut_image/Train'
test_dir = 'cut_image/Test'

In [6]:
# data image augmentation to create more variation of XRD patterns
train_data = ImageDataGenerator(rescale=1. / 255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

# only resize for test data
test_data = ImageDataGenerator(rescale=1. / 255)

In [7]:
# import train data consisting of 'binary' or  2 classifications (BCC or FCC) 
train_gen = train_data.flow_from_directory(train_dir, target_size=(img_width, img_height),
                                           batch_size=batch_size,
                                           class_mode='binary')

Found 1330 images belonging to 2 classes.


In [8]:
# import test data consisting of 'binary' or  2 classifications (BCC or FCC) 
test_gen = test_data.flow_from_directory(test_dir, target_size=(img_width, img_height),
                                          batch_size=batch_size,
                                           class_mode='binary')

Found 196 images belonging to 2 classes.


In [9]:
# build CNN model
model = Sequential()
# extract features by iterating across image
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(ZeroPadding2D((1,1)))
#convert 3D features to 1D feature vectors
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 377, 286, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 377, 286, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 188, 143, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 186, 141, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 186, 141, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 93, 70, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 91, 68, 64)      

In [10]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.fit_generator(train_gen,
                    steps_per_epoch=train_samples // batch_size,
                    epochs=epochs,
                    validation_data=test_gen,
                    validation_steps=test_samples // batch_size)


Epoch 1/75


In [54]:
BCC_test_dir = 'XRayDawgz/Images/Test/BCC_test/'
FCC_test_dir = 'XRayDawgz/Images/Test/FCC_test/'

BCC_Prediction = np.zeros(98)
FCC_Prediction = np.zeros(98)
i=0
correct_count=0
for file in os.listdir(BCC_test_dir):
  predict_img = image.load_img(BCC_test_dir + file, target_size = (img_width, img_height))
  predict_img = image.img_to_array(predict_img)
  predict_img = np.expand_dims(predict_img, axis = 0)
  result = model.predict(predict_img)
  pred = result.astype(int)
  if pred[0] is 0:
    correct_count += 1
  else:
    pass
print('BCC prediction accuracy: ' + str(correct_count // 98))

i=0
for file in os.listdir(FCC_test_dir):
  predict_img = image.load_img(FCC_test_dir + file, target_size = (img_width, img_height))
  predict_img = image.img_to_array(predict_img)
  predict_img = np.expand_dims(predict_img, axis = 0)
  result = model.predict(predict_img)
  FCC_Prediction[i] = result
  i += 1
print(FCC_Prediction)

BCC prediction accuracy: 0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1.]


In [20]:
predict_img = image.load_img('download.png', target_size = (img_width, img_height))
predict_img = image.img_to_array(predict_img)
predict_img = np.expand_dims(predict_img, axis = 0)

result = np.round(model.predict(predict_img), decimals=0)
print(np.round(result, decimals=0))

[[1.]]
